In [4]:
# we will try to predict the fixed point

# stap 0: select right models (hardcoded a bit)
# stap 1: aangezien option_1 de kleinere x-waarde heeft: beschouw alleen deze en gebruik deze als input voor option 1 en option 3
# stap 2: Newton's method to determine fixed point

"""
This part is still hardcoded, in:
-average_nullclines_from_modelnames()
the modelnames are typed within
"""


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from ast import literal_eval
from FitzHugh_Nagumo_ps import nullcline_and_boundary, nullcline_vdot, nullcline_wdot, limit_cycle
from Nullcine_MSE_plot import open_csv_and_return_all
from plot_NN_ps import retrieve_model_from_name, normalize_axis_values, reverse_normalization, search_5_best_5_worst_modelnames
from scipy.interpolate import interp1d
import seaborn as sns
from settings import TAU, A, B


In [12]:
# Newton-Raphson method
def newton_raphson_method(x, F, G):
    F_interp = interp1d(x, F, kind='linear')
    G_interp = interp1d(x, G, kind='linear')

    # Initial guess for the intersection point
    x0 = 0

    # Max iterations before stopping:
    max_iter = 200

    # Tolerance for convergence:
    tolerance = 1e-6

    # Newton's method iteration:
    for i in range(max_iter):
        # call H the difference between F and G

        # Evaluate H(x0) and H'(x0)
        H_val = F_min_G(x0, F_interp, G_interp)
        H_prime_val = F_min_G_prime(x0, F_interp, G_interp)
        
        # Update x0 using Newton's method
        x1 = x0 - H_val / H_prime_val
        
        # Check for convergence
        if abs(x1 - x0) < tolerance:
            print('Succesfully Acquired Wanted Accuracy')
            break
        
        # Update x0 for the next iteration
        x0 = x1

    print('The fixed point is located at', x0, F_interp(x0))
    return x0, F_interp(x0)

def F_min_G(x, F_interpol, G_interpolate):
    return F_interpol(x) - G_interpolate(x)

def F_min_G_prime(x, F_interpol, G_interpol, h=1e-6):
    return (F_min_G(x+h, F_interpol, G_interpol) - F_min_G(x, F_interpol, G_interpol)) / h

# => Searching for fixed point <=

#  Extra Functions

def return_partial_nullcline_from_modelname(modelname, title_extra='', plot_bool=True, df=None) -> tuple[np.ndarray, np.ndarray, pd.Series]:
    """ Returns the nullcline on the phase space, but in option_1 region of axis value
    df is a dataframe with one row containing everything

    input:
    title_extra:
        Something extra in to put at the end in the title, like 'low val, high MSE'.
    """

    if df is None:
        absolute_path = os.path.dirname(__file__)
        relative_path = f"FHN_NN_loss_and_model_{TAU}.csv"
        csv_name = os.path.join(absolute_path, relative_path)
        df = pd.read_csv(csv_name, converters={"nodes": literal_eval, "mean_std": literal_eval}) # literal eval returns [2,2] as list not as str

    option = df[(df['modelname'] == modelname)]['option'].iloc[0]
    mean_std = df[(df['modelname'] == modelname)]['mean_std'].iloc[0]
    # learning_rate = df[(df['modelname'] == modelname)]['learning_rate'].iloc[0]
    # nodes = df[(df['modelname'] == modelname)]['nodes'].iloc[0]
    # layers = df[(df['modelname'] == modelname)]['layers'].iloc[0]
    # max_epochs = df[(df['modelname'] == modelname)]['epoch'].iloc[-1]
    # normalization_method = df[(df['modelname'] == modelname)]['normalization_method'].iloc[0]
    # activation_function = df[(df['modelname'] == modelname)]['activation_function'].iloc[0]

    model = retrieve_model_from_name(modelname)

    # load data of nullclines in phasespace
    amount_of_points = 500
    axis_values_for_nullcline, exact_nullcline_values = nullcline_and_boundary('option_1', amount_of_points)

    # Predict normalized data 
    input_prediction, reverse_norm_mean_std = normalize_axis_values(axis_values_for_nullcline, mean_std, option)
    prediction_output_normalized = model.predict(input_prediction)
    # Reverse normalize to 'normal' data
    prediction_output_column = reverse_normalization(prediction_output_normalized, reverse_norm_mean_std)
    prediction_output = prediction_output_column.reshape(-1)
    
    return (axis_values_for_nullcline, prediction_output, df)

def select_5_best_validation(df, learning_rate, nodes, layers, max_epochs, option, amount):

    df = open_csv_and_return_all(option, learning_rate, max_epochs, nodes, layers, amount=20)

    df_sorted = df.sort_values(by=['validation'], ascending=True)

    cut_off = 5
    best_models = df_sorted.iloc[:5]['modelname'].tolist()

    return best_models


def save_mean_data(array, name):
    absolute_path = os.path.dirname(__file__)
    relative_path = "mean_data_predicted_nullcline"
    folder_path = os.path.join(absolute_path, relative_path)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    full_path = os.path.join(folder_path, name)
    np.save(full_path, array)


# Main Function:

def average_nullclines_from_modelnames(save):
    """
    Plot the mean nullcline of 5 models for each nullcline and visualize the predicted fixed point using Newton-Raphson method.

    Parameters:
        save (bool): Indicates whether to save the plotted data.

    Returns:
        None

    Notes:
        This function has been HARDCODED:
        This function retrieves predictions from 5 best models for 'option_3' and 'option_1'. 
        For 'option_3', models with specified hyperparameters are selected and nullcline predictions are calculated.
        For 'option_1', models with specified hyperparameters are selected based on validation performance and nullcline predictions are calculated.
        The mean predictions for both options are plotted, along with the predicted fixed point and real fixed point.
        Additionally, the function plots the limit cycle and nullclines for the system.
        The nullclines (option1 and option3) are only plotted in the 'option_1' region to match predictions.
    """

    #  Prediction for option_3
    best_worst_modelnames, _ = search_5_best_5_worst_modelnames(option='option_3', learning_rate=0.01, max_epochs=499, nodes=[16,16], layers=2, normalization_method='min-max', activation_function='relu')
    modelnames = best_worst_modelnames['best models']

    df = None

    all_predictions = np.zeros((len(modelnames),), dtype=object)
    for i, modelname in enumerate(modelnames):
        (axis_value, all_predictions[i], df) =  return_partial_nullcline_from_modelname(modelname, title_extra='', plot_bool=False, df = df)
    
    mean_prediction_option3 = np.mean(all_predictions, axis=0)
    plt.plot(axis_value, mean_prediction_option3, color='green', label='prediction option_3')
    if save:
        save_mean_data(mean_prediction_option3, 'best5_option_3_[16,16]relu_minmax_0.01_499_amount40')

    # Prediction for option_1
    modelnames = select_5_best_validation(df=df, learning_rate=0.01, nodes=[8,8], layers=2, max_epochs=99, option='option_1', amount=20)

    all_predictions = np.zeros((len(modelnames),), dtype=object)
    for i, modelname in enumerate(modelnames):
        (axis_value, all_predictions[i], df) =  return_partial_nullcline_from_modelname(modelname, title_extra='', plot_bool=False, df = df)
    
    mean_prediction_option1 = np.mean(all_predictions, axis=0)
    plt.plot(axis_value, mean_prediction_option1, color='purple', label='prediction option_3')
    if save:
        save_mean_data(mean_prediction_option1, 'best5_option_1_[8,8]_0.01_99_amount20')
        # did not include an 'open' feature yet

    xFP, yFP = newton_raphson_method(axis_value, mean_prediction_option1, mean_prediction_option3)
    plt.scatter([xFP], [yFP], label='predicted fixed point', color='red')

    x_real_FP = 0.40886584
    A = 0.7
    B = 0.8
    y_real_FP = (x_real_FP + A) / B
    plt.scatter([x_real_FP], [y_real_FP], color = 'b', marker='o', label='real fixed point')

    # Now plotting the limit cycle together with the (real) nullclines
    x_lc, y_lc = limit_cycle()
    plt.plot(x_lc, y_lc, 'r-', label=f'LC = {0}')
    # Plot Nullcines
    # vdot
    v = np.linspace(-2.5, 2.5, 1000)
    plt.plot(v, nullcline_vdot(v), '--', color = "lime", label = r"$w=v - (1/3)*v**3 + R * I$"+r" ,$\dot{v}=0$ nullcline")
    # wdot
    v = np.linspace(-2.5, 2.5, 1000)
    plt.plot(v, nullcline_wdot(v), '--', color = "cyan", label = r"$w=(v + A) / B$"+r" ,$\dot{w}=0$ nullcline")
    
    plt.xlabel('v (voltage)')
    plt.ylabel('w (recovery variable)')
    # plt.title(f'Predictions!!')
    plt.title(f"Phase Space: Limit Cycle and Nullclines with Predictions.\nBest 5 of validation.\n Option 1, lr0.01, 99, [8,8] #20\nOption 3, lr0.01, 499, [16,16] relu min-max #40")
    plt.grid(True)
    plt.legend(loc='upper left')
    plt.show()

def find_fixed_point_intersection_two_modelname(modelname_param1, modelname_param2, df):

    (axis_value, predictions1, df) =  return_partial_nullcline_from_modelname(modelname_param1, title_extra='', plot_bool=False, df = df)

    (axis_value, predictions2, df) =  return_partial_nullcline_from_modelname(modelname_param2, title_extra='', plot_bool=False, df = df)

    xFP, yFP = newton_raphson_method(axis_value, predictions1, predictions2)

    if xFP < min(axis_value) or xFP > max(axis_value):
        assert False, f'value xFP does not satisfy min(axis value)<xFP<max(axis value): {min(axis_value)} < {xFP} < {max(axis_value)}.'
    return xFP, yFP, df

def all_fixed_point_from_best_models(best_5_modelnames_param1, best_5_modelnames_param2, df):
    """
    Finds the fixed points of the 5 best models for each nullcline: so 25 points in total
    """

    fixed_point_x_value = []
    fixed_point_y_value = []
    for modelname1 in best_5_modelnames_param1:
        for modelname2 in best_5_modelnames_param2:
            
            x, y_arr, df = find_fixed_point_intersection_two_modelname(modelname1, modelname2, df)
            y = float(y_arr)
            fixed_point_x_value.append(x)
            fixed_point_y_value.append(y)

    return fixed_point_x_value, fixed_point_y_value

def fixed_point_analysis_gaussian_fit():

    df = None
    if df is None:
        # absolute_path = os.path.dirname(__file__)        
        absolute_path = os.path.abspath('')
        relative_path = f"FHN_NN_loss_and_model_{TAU}.csv"
        csv_name = os.path.join(absolute_path, relative_path)
        df = pd.read_csv(csv_name, converters={"nodes": literal_eval, "mean_std": literal_eval}) # literal eval returns [2,2] as list not as str

    best_worst_modelnames_param1, _ = search_5_best_5_worst_modelnames(option='option_3', learning_rate=0.01, max_epochs=499, nodes=[16,16], layers=2, normalization_method='min-max', activation_function='relu')
    best_val_modelnames_param1 = best_worst_modelnames_param1['best models']

    best_worst_modelnames_param2, _ = search_5_best_5_worst_modelnames(option='option_1', learning_rate=0.01, max_epochs=499, nodes=[16,16], layers=2, normalization_method='min-max', activation_function='relu')
    best_val_modelnames_param2 = best_worst_modelnames_param2['best models']

    print("All Models Found, starting to search for fixed point\n")
    x_values_FP, y_values_FP = all_fixed_point_from_best_models(best_val_modelnames_param1, best_val_modelnames_param2, df)
    print(x_values_FP, y_values_FP)

    x_values_sigmpoid_FP, y_values_sigmoid_FP = all_fixed_point_from_best_models('6bc4b512c04a4401b2eb80b1fb160461', best_val_modelnames_param2, df)

    return x_values_FP, y_values_FP, x_values_sigmpoid_FP, y_values_sigmoid_FP

def plot_fixed_points(x_values_FP, y_values_FP, x_values_sigmpoid_FP, y_values_sigmoid_FP):
    # x = [(0.2579361321410763, 1.1974153677670007), (0.2578986349102627, 1.1973898568549572), (0.2579480258962664, 1.1974234595790076), (0.2578934446236969, 1.1973863256890236),
    #     (0.2578982574320299, 1.197389600040953), (0.06747968602243959, 0.9593495252593581), (0.06745663612048293, 0.959331612039192), (0.06748983242644874, 0.9593574105323303),
    #     (0.06743880905589467, 0.9593177577445627), (0.06744678671768216, 0.9593239575805654), (0.4960329062995323, 1.495030779896996), (0.49597090995841725, 1.4949890501079361),
    #     (0.4960500537322715, 1.4950423218483153), (0.495980637494374, 1.4949955977205251), (0.49598330096963045, 1.4949973905079479), (0.21544334146371094, 1.1443004253029967),
    #     (0.2154103272407928, 1.1442787081894892), (0.21545401069428866, 1.1443074436384597), (0.21540408674743075, 1.1442746031253468), (0.21540897416753152, 1.144277818123239),
    #     (0.3706149332741673, 1.3382610442561138), (0.37043596011870455, 1.338066059102687), (0.37066557363020575, 1.3383162151981816), (0.37044627337859726, 1.3380772950478617),
    #     (0.37045823893476604, 1.3380903311135184)]

    # x_values_FP = [xFP for (xFP, yFP) in x]
    # y_values_FP = [yFP for (xFP, yFP) in x]

    data = pd.DataFrame({'X': x_values_FP, 'Y': y_values_FP})
    data['Fixed Point'] = 'Predicted fixed point'

    hue_color = {'Predicted fixed point': 'red'}
    g = sns.jointplot(data=data, x='X', y='Y', hue='Fixed Point', ratio=10, palette=hue_color)
    g.fig.set_size_inches((9.5,6))

    plt.xlabel("Validation")
    plt.ylabel("Mean Squared Error")

    x_mean = np.mean(x_values_FP)
    y_mean = np.mean(y_values_FP)

    plt.scatter([x_mean], [y_mean], color = 'green', marker='o', label='mean fixed point')

    plt.scatter([x_values_sigmpoid_FP], [y_values_sigmoid_FP], color= 'pink', marker='o', label='detailed fixed point')

    # x_std = np.std(x_values_FP)
    # y_std = np.std(y_values_FP)
    # confidence_ellipse = plt.Rectangle((x_mean-x_std, y_mean-y_std), 
    #                                 2*x_std, 2*y_std, 
    #                                 edgecolor='b', facecolor='none', linestyle='--', label='95% CI')
    # plt.gca().add_patch(confidence_ellipse)
    # using distances
    # plt.annotate('(', (x, y), xytext=(-10, 10), textcoords='offset points', fontsize=12, rotation=0)
    # plt.annotate(')', (x_point - distance, y_point - distance * slope), xytext=(-10, 10), textcoords='offset points', fontsize=12, rotation=0)


    # Phase Space Standard
    x_real_FP = 0.40886584
    A = 0.7
    B = 0.8
    y_real_FP = (x_real_FP + A) / B
    plt.scatter([x_real_FP], [y_real_FP], color = 'b', marker='o', label='real fixed point')

    # Print Standard Deviation between Predicted FP and real FP
    points = [(x,y) for x,y in zip(x_values_FP, y_values_FP)]
    std_distance = distance_deviation_calculator(x_mean, y_mean, points=points)
    std_distance_mean_from_real = np.sqrt((x_real_FP-x_mean)**2 + (y_real_FP-y_mean)**2) / std_distance
    print(f"\nOur mean point ({round(x_mean,2)}, {round(y_mean,2)}) is distance {round(std_distance_mean_from_real,2)}stds from real point ({round(x_real_FP,2)}, {round(y_real_FP,2)})")


    # Now plotting the limit cycle together with the (real) nullclines
    x_lc, y_lc = limit_cycle()
    plt.plot(x_lc, y_lc, 'r-', label=f'LC')
    # Plot Nullcines
    # vdot
    v = np.linspace(-2.5, 2.5, 1000)
    plt.plot(v, nullcline_vdot(v), '--', color = "lime", label = r"$w=v - (1/3)*v**3 + R * I$"+r" ,$\dot{v}=0$")
    # wdot
    v = np.linspace(-2.5, 2.5, 1000)
    plt.plot(v, nullcline_wdot(v), '--', color = "cyan", label = r"$w=(v + A) / B$"+r" ,$\dot{w}=0$")
    
    plt.xlabel('v (voltage)')
    plt.ylabel('w (recovery variable)')
    # plt.title(f'Predictions!!')
    plt.suptitle(f"Phase Space: Limit Cycle and Nullclines with Predictions.\nBest 5 of validation.\n Option 1, lr0.01, 499, [16,16] relu min-max #40\nOption 3, lr0.01, 499, [16,16] relu min-max #40")
    plt.grid(True)
    plt.legend(bbox_to_anchor=(1.10, 1.0), loc='upper left')

    plt.subplots_adjust(top=0.85, right=0.7) # Reduce plot to make room 

    plt.show()
    
def distance_deviation_calculator(x_mean, y_mean, points):
    distance = []
    for (x,y) in points[0::5]:
        print(x,y)
        calculated_distance = np.sqrt((x-x_mean)**2 + (y-y_mean)**2)
        distance.append(calculated_distance)
    return np.std(distance)


In [13]:
if __name__ == '__main__':
    # average_nullclines_from_modelnames(save=False)

    x_values_FP, y_values_FP, x_values_sigmpoid_FP, y_values_sigmoid_FP = fixed_point_analysis_gaussian_fit()


SyntaxError: unexpected EOF while parsing (<unknown>, line 1)

In [ ]:
if __name__ == '__main__':
    
    plot_fixed_points(x_values_FP, y_values_FP, x_values_sigmpoid_FP, y_values_sigmoid_FP)